# Raw Data Exploration

This notebook performs an initial exploration of the raw COVID Tracking Project datasets. The goal is to understand the data structure, identify key columns, and prepare for cleaning and integration.

In [1]:
import pandas as pd
import os

cwd = os.getcwd()
if cwd.endswith("notebooks"):
    os.chdir("..")

In [2]:
states_current = pd.read_csv('data/raw/states_current.csv')
states_daily = pd.read_csv('data/raw/states_daily.csv')
us_current = pd.read_csv('data/raw/us_current.csv')
us_daily = pd.read_csv('data/raw/us_daily.csv')
states_info = pd.read_csv('data/raw/states_info.csv')

print(f"Current States data loaded: {len(states_current)} records")
print(f"Daily States data loaded: {len(states_daily)} records")
print(f"Current US data loaded: {len(us_current)} records")
print(f"Daily US data loaded: {len(us_daily)} records")
print(f"States Info data loaded: {len(states_info)} records")

Current States data loaded: 56 records
Daily States data loaded: 20780 records
Current US data loaded: 1 records
Daily US data loaded: 420 records
States Info data loaded: 56 records


**Note:** Our main focus will be on `states_daily` and `us_daily`, which contain state-level and national-level daily COVID-19 statistics.

## Explore `states_daily`

In [3]:
states_daily.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [4]:
# Basic info and structure
states_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20780 entries, 0 to 20779
Data columns (total 56 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   date                         20780 non-null  int64  
 1   state                        20780 non-null  object 
 2   positive                     20592 non-null  float64
 3   probableCases                9271 non-null   float64
 4   negative                     13290 non-null  float64
 5   pending                      2138 non-null   float64
 6   totalTestResultsSource       20780 non-null  object 
 7   totalTestResults             20614 non-null  float64
 8   hospitalizedCurrently        17339 non-null  float64
 9   hospitalizedCumulative       12382 non-null  float64
 10  inIcuCurrently               11636 non-null  float64
 11  inIcuCumulative              3789 non-null   float64
 12  onVentilatorCurrently        9126 non-null   float64
 13  onVentilatorCumu

In [5]:
# Check for date range
print(f"Date range: {states_daily['date'].min()} to {states_daily['date'].max()}")

Date range: 20200113 to 20210307


In [6]:
# Check for duplicates
states_daily.duplicated(['date', 'state']).sum()

np.int64(0)

In [7]:
# Check for missing values
states_daily.isna().sum().sort_values(ascending=False)

grade                          20780
dataQualityGrade               20780
positiveTestsPeopleAntigen     20147
negativeTestsPeopleAntibody    19808
totalTestsPeopleAntigen        19781
positiveTestsPeopleAntibody    19686
onVentilatorCumulative         19490
negativeTestsAntibody          19322
pending                        18642
totalTestsPeopleAntibody       18580
positiveTestsAntigen           18547
hospitalizedDischarged         17710
positiveTestsAntibody          17434
totalTestsAntigen              17359
inIcuCumulative                16991
totalTestsAntibody             15991
negativeTestsViral             15756
totalTestEncountersViral       15549
deathProbable                  13187
positiveTestsViral             11822
onVentilatorCurrently          11654
totalTestsPeopleViral          11583
probableCases                  11509
deathConfirmed                 11358
inIcuCurrently                  9144
recovered                       8777
hospitalized                    8398
h

### Key Exploration Findings  

1. Massive Missing Data: A large number of columns are unusable. `grade` and `dataQualityGrade` are 100% null and provide no information. Other columns, while potentially useful, have very high rates of missing data. Key metrics like `hospitalizedCurrently` and `hospitalizedCumulative` also have thousands of missing entries.

2. Data Type Issues: The primary `date` column is stored as an integer rather than a proper datetime object. This is unsuitable for time-series analysis. Other columns like `lastUpdateEt` are similarly stored as objects and need conversion.

3. Non-Informative & Deprecated Columns: Several columns are deprecated or non-informative for analysis. These include scoring columns (`commercialScore`, `negativeScore`, `positiveScore`, `score`), data quality grades (`grade`, `dataQualityGrade`), and metadata fields (`hash`, `checkTimeEt`). These columns should be removed.

The next step is to move on to data cleaning and preprocessing:

- Drop deprecated and non-informative columns to simplify the dataset.
- Convert the `date` and `lastUpdateEt` columns to proper datetime objects to enable time-series operations.
- Handle missing data by filling numerical fields by using forward-filling for current fields and enforce logic for cumulative fields. Remaining nulls in numeric columns will be filled with 0.
- Add new metrics for analysis, such as growth rates and key ratios, which are not present in the raw data.

## Explore `us_daily`

In [ ]:
us_daily.head()

,date,states,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,lastModified,recovered,total,posNeg,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease,hash
0,20210307,56,28756489.0,74582825.0,11808.0,40199.0,776361.0,8134.0,45475.0,2802.0,...,2021-03-07T24:00:00Z,NaN,0,0,842,726,131835,41835,1170059,a80d0063822e251249fd9a44730c49cb23defd83
1,20210306,56,28714654.0,74450990.0,11783.0,41401.0,775635.0,8409.0,45453.0,2811.0,...,2021-03-06T24:00:00Z,NaN,0,0,1680,503,143835,60015,1430992,dae5e558c24adb86686bbd58c08cce5f610b8bb0
2,20210305,56,28654639.0,74307155.0,12213.0,42541.0,775132.0,8634.0,45373.0,2889.0,...,2021-03-05T24:00:00Z,NaN,0,0,2221,2781,271917,68787,1744417,724844c01659d0103801c57c0f72bf8cc8ab025c
3,20210304,56,28585852.0,74035238.0,12405.0,44172.0,772351.0,8970.0,45293.0,2973.0,...,2021-03-04T24:00:00Z,NaN,0,0,1743,1530,177957,65487,1590984,5c549ad30f9abf48dc5de36d20fa707014be1ff3
4,20210303,56,28520365.0,73857281.0,11778.0,45462.0,770821.0,9359.0,45214.0,3094.0,...,2021-03-03T24:00:00Z,NaN,0,0,2449,2172,267001,66836,1406795,fef6c425d2b773a9221fe353f13852f3e4a4bfb0


In [ ]:
# Basic info and structure
us_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      420 non-null    int64  
 1   states                    420 non-null    int64  
 2   positive                  419 non-null    float64
 3   negative                  372 non-null    float64
 4   pending                   369 non-null    float64
 5   hospitalizedCurrently     356 non-null    float64
 6   hospitalizedCumulative    369 non-null    float64
 7   inIcuCurrently            347 non-null    float64
 8   inIcuCumulative           348 non-null    float64
 9   onVentilatorCurrently     348 non-null    float64
 10  onVentilatorCumulative    341 non-null    float64
 11  dateChecked               420 non-null    object 
 12  death                     392 non-null    float64
 13  hospitalized              369 non-null    float64
 14  totalTestR

In [ ]:
# Check for date range
print(f"Date range: {us_daily['date'].min()} to {us_daily['date'].max()}")

Date range: 20200113 to 20210307


In [ ]:
# Check for duplicates
us_daily.duplicated(['date', 'states']).sum()

np.int64(0)

In [ ]:
# Check for missing values
us_daily.isna().sum().sort_values(ascending=False)

recovered                   420
onVentilatorCumulative       79
inIcuCurrently               73
onVentilatorCurrently        72
inIcuCumulative              72
hospitalizedCurrently        64
pending                      51
hospitalizedCumulative       51
hospitalized                 51
negative                     48
death                        28
positive                      1
totalTestResultsIncrease      0
positiveIncrease              0
posNeg                        0
negativeIncrease              0
hospitalizedIncrease          0
deathIncrease                 0
date                          0
total                         0
lastModified                  0
totalTestResults              0
states                        0
dateChecked                   0
hash                          0
dtype: int64

### Key Exploration Findings  

1. Significant Missing Data: Several key fields contain substantial missing values. Metrics such as `hospitalizedCurrently`, `inIcuCurrently`, and `onVentilatorCurrently` each have over 60 missing entries, while cumulative fields like `hospitalizedCumulative` and `inIcuCumulative` also contain dozens of nulls. The `recovered` column is 100% null and provides no usable information.  

2. Data Type Issues: The primary `date` column is stored as an integer rather than a proper datetime object, making it unsuitable for time-series analysis. Additional timestamp-related fields, such as `dateChecked` and `lastModified`, are stored as generic objects.  

3. Non-Informative & Deprecated Columns: Several columns add no analytical value or are redundant. These include metadata fields (`hash`, `lastModified`, `dateChecked`), redundant totals (`total`, `posNeg`), and the fully-null `recovered` column. These fields should be removed to streamline the dataset.  

The next step is to move on to data cleaning and preprocessing:  

- Drop deprecated and non-informative columns to simplify the dataset.  
- Convert the `date` column to a proper datetime object and sort the dataset chronologically.  
- Handle missing data by forward-filling current hospitalization fields and enforcing logical behavior for cumulative metrics using `cummax`. Remaining nulls in numeric columns will be filled with 0.  
- Recompute daily increases and correct inconsistencies in testing data, such as ensuring `totalTestResults` is never less than `positive + negative`.  
- Add new analytical metrics, including growth rates and ratio-based indicators, which are not present in the raw dataset.  
